# Setup for Test the knowledge of LLMs of common idioms (list form https://englishlovers.in/what-is-idiom-examples-definition-list-of-1000-idiom/)

In [ ]:
# %pip install -U "ell-ai[all]"
# %pip install pandas
# %pip install python-dotenv
# %pip install ollama
#%pip install -U scikit-learn

[notice] A new release of pip is available: 24.0 -> 24.3.1

[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)

In [26]:
#imports and setup
import pandas as pd
import ell
from pydantic import BaseModel, Field
import csv
from dotenv import load_dotenv
# import openai
import ollama
from sklearn.metrics.pairwise import cosine_similari

load_dotenv()

False

In [84]:
# read in the prepared csv file
df = pd.read_csv('../data/idioms.csv', delimiter=':', skiprows=1, names=['idiom', 'meaning'])	

print(df.head())
print(df.shape)

                             idiom  \
0                       A Bit Much   
1             A Bite at The Cherry   
2                       A Busy Bee   
3             A Cat Has Nine Lives   
4  A Cat in Gloves Catches No Mice   

                                             meaning  
0            More than is reasonable; a bit too much  
1   A good opportunity that isn’t available to ev...  
2   A busy, active person who moves quickly from ...  
3        Cats seem to get away with dangerous things  
4   You can’t get what you need if you’re too car...  


(1450, 2)


In [9]:
# prepare function for idiom translation using ell with specified output format 

prompt = "Translate the following idiom, or phrase, into the most concise and accurate translation possible only include a single answer:"


# model with OpenAI that type safety / structured outputs needs api key and funding

# class IdiomTranslation(BaseModel):
#     generatedTranslation: str = Field(description="The translation of the presented idiom as concise and accurate as possible")

# @ell.complex(model="gpt-4o-mini", response_format=IdiomTranslation, temperature=0)
# def translate_idiom(idiom: str) -> IdiomTranslation:
#     return [ell.system(prompt), ell.user(idiom)]

# run with local ollama
# Use models automatically registered by asking ollama
ell.models.ollama.register(base_url="http://localhost:11434/v1")

ell.init(store='./logdir', autocommit=True, autocommit_model="llama3.1:8b", verbose=True)


@ell.complex(model="llama3.1:8b", temperature=0)  # adding response format leads to validation errors
def translate_idiom(idiom: str):
    return [ell.system(prompt), ell.user(idiom)]


In [13]:
# test

idiom = df['idiom'][0]
print(f"Idiom: {idiom}")

response = translate_idiom(idiom)
print(f"Translation: {response.content[0].text}")

print(f"Answer form the csv: {df['meaning'][0]}")

Idiom: A Bit Much
╔══════════════════════════════════════════════════════════════════════════════╗
║ translate_idiom(A Bit Mu..)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│      system: Translate the following idiom, or phrase, into the most
│              concise and accurate translation possible only include a
│              single answer:
│
│        user: A Bit Much
╟──────────────────────────────────────────────────────────────────────────────╢
║ Output:
╟──────────────────────────────────────────────────────────────────────────────╢
│   assistant: Too much to handle.
╚══════════════════════════════════════════════════════════════════════════════╝
Translation: Too much to handle.
Answer form the csv:  More than is reasonable; a bit too much


In [30]:
# Evaluate the Translation of the model using similarity score of embeddings
from sklearn.metrics.pairwise import cosine_similarity

embedding_translation = ollama.embeddings(model="nomic-embed-text", prompt=response.content[0].text)
embedding_real_answer = ollama.embeddings(model="nomic-embed-text", prompt=df['meaning'][0])

# Reshape the embeddings to 2D arrays
embedding_translation_reshaped = [embedding_translation.embedding]
embedding_real_answer_reshaped = [embedding_real_answer.embedding]

similarity_score = cosine_similarity(embedding_translation_reshaped, embedding_real_answer_reshaped)
print(f"Similarity score: {similarity_score[0][0]}")

Similarity score: 0.7029551047916202


In [32]:
# define a function to evaluate a model (translate all idioms and compare to the real answers for a given model)
def evaluate_model(df, model):
    scores = []
    # counter = 0
    for i in range(df.shape[0]):
        # counter += 1
        idiom = df['idiom'][i]
        response = model(idiom)
        embedding_translation = ollama.embeddings(model="nomic-embed-text", prompt=response.content[0].text)
        embedding_real_answer = ollama.embeddings(model="nomic-embed-text", prompt=df['meaning'][i])
        embedding_translation_reshaped = [embedding_translation.embedding]
        embedding_real_answer_reshaped = [embedding_real_answer.embedding]
        similarity_score = cosine_similarity(embedding_translation_reshaped, embedding_real_answer_reshaped)
        scores.append(similarity_score[0][0])
        # if counter == 20:
        #     break
    return scores

In [ ]:
# test evaluate_model
ell.init(verbose=False)
scores = evaluate_model(df, translate_idiom)
print(scores)

[np.float64(0.7029551047916202), np.float64(0.6301009567222098), np.float64(0.5834982262759125), np.float64(0.42137171566679565), np.float64(0.5331034253454627), np.float64(0.7661584280519161), np.float64(0.5916024939260345), np.float64(0.7214854138672798), np.float64(0.532905051594004), np.float64(0.6488030518668693), np.float64(0.6209738865324419), np.float64(0.7677581845446777), np.float64(0.4325530505028463), np.float64(0.4366071028157236), np.float64(0.5909186913806793), np.float64(0.6765920221895259), np.float64(0.48033139946408177), np.float64(0.7600456465792751), np.float64(0.5203125677867839), np.float64(0.6635981714647708)]


In [70]:
# write the results to a csv file
def write_translations_to_csv(df, model, filename='translations.csv'):
    header = ['idiom', 'meaning', 'model', 'response']
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for i in range(df.shape[0]):
            idiom = df['idiom'][i]
            response = model(idiom)
            writer.writerow([idiom, df['meaning'][i], 'llama3.1:8b', response.content[0].text])

def calculate_similarity_scores_and_write_to_csv(df, filename='translations_with_similarity_scores.csv'):
    header = ['idiom', 'meaning', 'response', 'similarity_score']
    df_translations = pd.read_csv('translations.csv', skiprows=1, names=['idiom', 'meaning','model', 'response'])
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for i in range(df.shape[0]):
            idiom = df_translations.iloc[i]['idiom']
            response = df_translations.iloc[i]['response']
            embedding_translation = ollama.embeddings(model="nomic-embed-text", prompt=response)
            embedding_real_answer = ollama.embeddings(model="nomic-embed-text", prompt=df['meaning'][i])
            embedding_translation_reshaped = [embedding_translation.embedding]
            embedding_real_answer_reshaped = [embedding_real_answer.embedding]
            similarity_score = cosine_similarity(embedding_translation_reshaped, embedding_real_answer_reshaped)
            writer.writerow([idiom, df['meaning'][i], response, similarity_score[0][0]])

In [86]:
# test eval_and_write
# write_translations_to_csv(df, translate_idiom, 'translations.csv')
calculate_similarity_scores_and_write_to_csv(df, 'translations_with_similarity_scores.csv')

In [87]:
# see basic statistics of the similarity scores
df_similarity_scores = pd.read_csv('translations_with_similarity_scores.csv', skiprows=1, names=['idiom', 'meaning', 'response', 'similarity_score'])
print(df_similarity_scores['similarity_score'].describe())

count    1450.000000
mean        0.660420
std         0.132097
min         0.319372
25%         0.565719
50%         0.657454
75%         0.753543
max         1.000000
Name: similarity_score, dtype: float64


In [98]:
# have a look at some entries with very high similarity scores 
# change display settings to show the full text
pd.set_option('display.max_colwidth', None)
print(df_similarity_scores[df_similarity_scores['similarity_score'] > 0.9].head())


                 idiom  \
17         A Home Bird   
55     Ace In The Hole   
98   All Hands on Deck   
125         All Thumbs   
127            All Wet   

                                                              meaning  \
17    Somebody who prefers to spend his social and free time at home.   
55                                                 A hidden advantage   
98                                                Everyone must help.   
125                                                            Clumsy   
127                                               Completely mistaken   

                                                     response  \
17   A person who prefers to stay at home rather than go out.   
55                                        A hidden advantage.   
98                               "Everyone's help is needed."   
125                                                   Clumsy.   
127                             Completely mistaken or wrong.   

     similari

In [101]:
# have a look at some entries with very low similarity scores
print(df_similarity_scores[df_similarity_scores['similarity_score'] < 0.4].head())

                   idiom  \
56   Ace Up One�s Sleeve   
103     All in One Piece   
131         Amber Nectar   
182           Baby Blues   
374  Come Rain and Shine   

                                                  meaning  \
56    A surprise advantage of which others are not aware.   
103                                                Safely   
131                                                  Beer   
182                                            Blue eyes.   
374              Do regularly, whatever the circumstances   

                    response  similarity_score  
56             A trump card.          0.396890  
103            "Whole cloth"          0.390095  
131            Honey of Gold          0.393247  
182   Postpartum depression.          0.319372  
374  Through thick and thin.          0.393893  
